In [72]:
from keras import *
from sklearn.model_selection import train_test_split
import pandas as pd
from hyperopt import hp,fmin,STATUS_OK ,tpe , Trials 
from sklearn.metrics import mean_squared_error
import mlflow as mf


In [66]:
mf.set_tracking_uri("http://127.0.0.1:5000")

In [46]:
data = pd.read_csv("winequality-white.csv" , sep=";")
data.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [47]:
target = data["quality"].values
dep_feats = data.iloc[: , :-1]
print(dep_feats.head(3) , target[:3])

   fixed acidity  volatile acidity  citric acid  ...    pH  sulphates  alcohol
0            7.0              0.27         0.36  ...  3.00       0.45      8.8
1            6.3              0.30         0.34  ...  3.30       0.49      9.5
2            8.1              0.28         0.40  ...  3.26       0.44     10.1

[3 rows x 11 columns] [6 6 6]


In [48]:
trainx,testx,trainy,testy = train_test_split(dep_feats , target, test_size=.3 , random_state=42)
print(trainx.shape , trainy.shape , testx.shape ,testy.shape)

(3428, 11) (3428,) (1470, 11) (1470,)


In [49]:
train_x , test_x , train_y , test_y = train_test_split(trainx , trainy , test_size=.3 , random_state=42)

In [186]:
train_x.shape , test_x.shape , dep_feats.shape[1]
import numpy as np
mf.tensorflow.autolog()


In [204]:
def create_and_train(batch , epoch , params , trainx , trainy , testx  ,testy , train_x , train_y):

    mean = np.mean(dep_feats,axis=0)
    var = np.var(dep_feats , axis=0)
    model = Sequential(
      [
        layers.Input(shape=(dep_feats.shape[1],)),
        layers.Normalization(mean=mean , variance=var),
        layers.Dense(124 , activation="relu"),
        layers.Dense(64,activation="relu"),
        layers.Dense(32,activation="relu"),
        layers.Dense(1)
      ]
    )

    model.compile(optimizer=optimizers.SGD(learning_rate=  params["lr"]  , momentum = params["momentum"]) , loss="mean_squared_error" ,
                  metrics=[metrics.RootMeanSquaredError()])
    
    with mf.start_run(nested=True):
        
        
        model.fit(train_x , train_y , batch_size=batch , epochs=epoch , validation_data=(testx , testy))

        evaluation_result = model.evaluate(testx, testy , batch_size = batch)
        rmse = evaluation_result[1]

        mf.log_params(params)
        mf.log_metric("rmse" , rmse)
        signature = mf.models.infer_signature(train_x, model.predict(train_x))
        mf.tensorflow.log_model(model , "model" )

        return {"loss":rmse , "model":model , "status":STATUS_OK}
    




In [205]:
def objective(params):
    result = create_and_train(
        batch=64,
        epoch=3,
        params=params,
        trainx=trainx,
        trainy=trainy,
        testx=testx,
        testy=testy,
        train_x=train_x,
        train_y=train_y
    )
    return result

In [206]:
import numpy as np
search_space = {
    "lr":hp.loguniform("lr" , np.log(1e-5) , np.log(1e-1) ),
    "momentum":hp.uniform("momentum",0,1)
}

In [207]:
mf.set_experiment("/wine_quality")

with mf.start_run():
    trials = Trials()

    best = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )
    best_run = sorted(trials.results , key=lambda x:x["loss"])[0]
    mf.log_params(best)
    mf.log_metric("final_rmse",best_run["loss"])
    signature = mf.models.infer_signature(train_x,best_run["model"].predict(train_x))
    mf.tensorflow.log_model(best_run["model"],"model")

    

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

2025/02/15 22:54:29 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'pandas.core.frame.DataFrame'>. Dataset logging skipped.

2025/02/15 22:54:29 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'DataFrame' object has no attribute 'flatten'



Epoch 1/3                                            

 1/38 ━━━━━━━━━━━━━━━━━━━━ 16s 445ms/step - loss: 38.5718 - root_mean_squared_error: 6.2106
  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 16.3314 - root_mean_squared_error: 3.9015 - val_loss: 0.9434 - val_root_mean_squared_error: 0.9713

Epoch 2/3                                            

 1/38 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.7971 - root_mean_squared_error: 0.8928
36/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8429 - root_mean_squared_error: 0.9170 
  0%|          | 0/4 [00:01<?, ?trial/s, best loss=?]

38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.8345 - root_mean_squared_error: 0.9124 - val_loss: 0.6489 - val_root_mean_squared_error: 0.8055

Epoch 3/3                                            

 1/38 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.7102 - root_mean_squared_error: 0.8427
  0%|          | 0/4 [00:01<?, ?trial/s, best loss=?]

...

 1/23 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.5782 - root_mean_squared_error: 0.7604
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5511 - root_mean_squared_error: 0.7423 

 1/75 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step               
72/75 ━━━━━━━━━━━━━━━━━━━━ 0s 730us/step     
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step      

 75%|███████▌  | 3/4 [01:19<00:23, 23.12s/trial, best loss: 0.7697697281837463]

2025/02/15 22:55:48 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.

2025/02/15 22:55:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



🏃 View run valuable-quail-26 at: http://127.0.0.1:5000/#/experiments/766684217518994152/runs/4a7fb325b0e1438285e2f415736af690

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/766684217518994152  

100%|██████████| 4/4 [01:27<00:00, 21.86s/trial, best loss: 0.7359983921051025]
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 999us/step


2025/02/15 22:55:57 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2025/02/15 22:56:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run unique-chimp-493 at: http://127.0.0.1:5000/#/experiments/766684217518994152/runs/84627afdac464e65bdb831a9c7f39a90
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/766684217518994152
